In [42]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [43]:
import warnings
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import (
    OneHotEncoder
)

from feature_engine.encoding import RareLabelEncoder
from sklearn.compose import ColumnTransformer
from feature_engine.datetime import DatetimeFeatures

import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from utils import eda_helper_functions
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.ensemble import IsolationForest
import sklearn

from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    MinMaxScaler,
    PowerTransformer,
    FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
    RareLabelEncoder,
    MeanEncoder,
    CountFrequencyEncoder
)

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

import warnings
pd.set_option('display.max_columns',None)
sklearn.set_config(transform_output = "pandas")
warnings.filterwarnings("ignore")

In [44]:
dtoj_transformer = Pipeline(steps = [
    ("dt",DatetimeFeatures(features_to_extract = ['weekend'],yearfirst = True,format = "mixed"))
])

In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.feature_utils import (
    is_north, find_part_of_month, part_of_day,
    make_month_object, remove_duration, have_info,
    duration_category,is_same_region
)
from utils.rbf import RBFPercentileSimilarity,RouteCreator


In [ ]:
import joblib

load_path = os.path.join("..", "artifacts", "latest_column_transformer.joblib")

# Load the column transformer
column_transformer = joblib.load(load_path)


print("✅ latest_column_transformer loaded successfully.")


✅ latest_column_transformer loaded successfully.


In [49]:
X_train_transformed = pd.read_csv("../transformed_data/X_train_transformed.csv")
X_test_transformed = pd.read_csv("../transformed_data/X_test_transformed.csv")
X_val_transformed = pd.read_csv("../transformed_data/X_val_transformed.csv")

y_train = pd.read_csv("../data/processed/train_data.csv").price
y_test = pd.read_csv("../data/processed/test_data.csv").price
y_val = pd.read_csv("../data/processed/val_data.csv").price

### Model Tuning

In [50]:
# Taking the XGBoost Model 

import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib  # for saving model

def objective(trial):
    # Suggest hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "random_state": 42,
        "n_jobs": -1
    }

    # Model
    model = XGBRegressor(**param)

    # Fit on training data
    model.fit(X_train_transformed, y_train)

    # Predict on tuning set
    preds = model.predict(X_val_transformed)

    # Evaluation metric
    mse = mean_squared_error(y_val, preds)
    return mse  # Minimize MSE


In [51]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100) 

[I 2026-01-12 20:21:28,493] A new study created in memory with name: no-name-09f94d73-767b-4bc5-8f26-bfbc5781cb3e
[I 2026-01-12 20:21:33,541] Trial 0 finished with value: 2858833.0 and parameters: {'n_estimators': 962, 'max_depth': 13, 'learning_rate': 0.15671218546148116, 'subsample': 0.8111279616467473, 'colsample_bytree': 0.9357495899116444, 'gamma': 9.184948020151854, 'min_child_weight': 9, 'reg_alpha': 2.5542404630983775, 'reg_lambda': 8.80416486159373}. Best is trial 0 with value: 2858833.0.
[I 2026-01-12 20:21:45,628] Trial 1 finished with value: 2784497.0 and parameters: {'n_estimators': 828, 'max_depth': 15, 'learning_rate': 0.17031195301546082, 'subsample': 0.8958261824494502, 'colsample_bytree': 0.7387152728221379, 'gamma': 8.09181555009947, 'min_child_weight': 1, 'reg_alpha': 2.225719563275047, 'reg_lambda': 4.526753850325793}. Best is trial 1 with value: 2784497.0.
[I 2026-01-12 20:21:50,811] Trial 2 finished with value: 2841921.25 and parameters: {'n_estimators': 725, 'ma

In [52]:
best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

final_model = XGBRegressor(**best_params)
final_model.fit(X_train_transformed, y_train)

model_dir = os.path.join("..", "artifacts", "models")
os.makedirs(model_dir, exist_ok=True)

# Full save path for model
model_path = os.path.join(model_dir, "xgb_flight_price_model.joblib")

# Save the model
joblib.dump(final_model, model_path)

print(f"✅ Final model trained and saved as {model_path}")

✅ Final model trained and saved as ..\artifacts\models\xgb_flight_price_model.joblib


### Loading the model

In [53]:
import joblib
models_path = os.path.join("..", "artifacts", "models")
xgboostmodel_path = os.path.join(models_path,"xgb_flight_price_model.joblib")
final_model = joblib.load(xgboostmodel_path)
print("✅ Model loaded successfully.")

✅ Model loaded successfully.


### Checking performance on Test data 

In [54]:
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error

y_test_pred = final_model.predict(X_test_transformed)

print("Test MSE:", mean_squared_error(y_test, y_test_pred))
print("Test MAE:", mean_absolute_error(y_test, y_test_pred))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))
print("Test R²:", r2_score(y_test, y_test_pred))

Test MSE: 3121326.75
Test MAE: 783.2699584960938
Test RMSE: 1766.727695486772
Test R²: 0.8662093877792358


### Checking if the model is overfitting or not

In [55]:
# First on training data

y_train_pred = final_model.predict(X_train_transformed)

print("Train MSE:", mean_squared_error(y_train, y_train_pred))
print("Train MAE:", mean_absolute_error(y_train, y_train_pred))
print("Train RMSE:", np.sqrt(mean_squared_error(y_train, y_train_pred)))
print("Train R²:", r2_score(y_train, y_train_pred))

Train MSE: 654963.0625
Train MAE: 355.0643005371094
Train RMSE: 809.2978824264895
Train R²: 0.969047486782074


In [56]:
# On validation data

y_val_pred = final_model.predict(X_val_transformed)

print("Val MSE:", mean_squared_error(y_val, y_val_pred))
print("Val MAE:", mean_absolute_error(y_val, y_val_pred))
print("Val RMSE:", np.sqrt(mean_squared_error(y_val, y_val_pred)))
print("Val R²:", r2_score(y_val, y_val_pred))

Val MSE: 2406418.75
Val MAE: 785.554931640625
Val RMSE: 1551.263597845318
Val R²: 0.8788323402404785


In [57]:
cleaned_flight_data = pd.read_csv("../data/processed/cleaned_flight_data.csv")

In [58]:
cleaned_flight_data["date"] = pd.to_datetime(cleaned_flight_data.rename(columns={'dtoj_year': 'year', 
                                     'dtoj_month': 'month', 
                                     'dtoj_day': 'day'})[['year', 'month', 'day']])
weekend = dtoj_transformer.fit_transform(cleaned_flight_data[['date']])
cleaned_flight_data = cleaned_flight_data.assign(is_weekend = weekend)
cleaned_flight_data.drop(columns = ['dep_time_min','dtoj_year'],inplace = True)

X = cleaned_flight_data.drop(columns = ['price'])
y = cleaned_flight_data.price

In [59]:
cleaned_flight_data.head(2)

,airline,source,destination,duration,total_stops,additional_info,price,dep_time_hour,dtoj_day,dtoj_month,date,is_weekend
0,Indigo,banglore,new delhi,170,0,no info,3897,22,24,3,2019-03-24,1
1,Air India,kolkata,banglore,445,2,no info,7662,5,1,5,2019-05-01,0


In [60]:
import joblib

# Define the path to the column transformer
load_path = os.path.join("..", "artifacts", "latest_column_transformer.joblib")

# Load the column transformer
column_transformer = joblib.load(load_path)


print("✅ latest column_transformer loaded successfully.")


✅ latest column_transformer loaded successfully.


In [61]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline

# Suppose you already have:
# column_transformer (your preprocessing steps)
# best_model (your tuned ML model, e.g. RandomForest with best params)

# Build pipeline = preprocessing + tuned model
final_pipeline = Pipeline(steps=[
    ('preprocess', column_transformer),
    ('model', final_model)
])

# Define KFold
kfold = KFold(n_splits=5, shuffle=True)

# Run CV
scores = cross_val_score(
    final_pipeline, 
    X,  # full cleaned raw data (without manual splitting)
    y, 
    cv=kfold, 
    scoring='r2'
)

print("Cross-validation R² scores:", scores)
print("Mean R²:", scores.mean())
print("Std Dev:", scores.std())


Cross-validation R² scores: [0.8630324  0.85595214 0.8634938  0.8561188  0.83122551]
Mean R²: 0.8539645314216614
Std Dev: 0.011821062362917812


In [62]:
# Suppose y_true and y_pred are arrays
df = pd.DataFrame({"y_true": y_test, "y_pred": y_test_pred})
df["error"] = abs(df["y_true"] - df["y_pred"])
df["perc_error"] = df["error"] / df["y_true"]

# Price bins (quartiles or custom ranges)
bins = [0, 5000, 10000, 20000, 80000]
labels = ["Budget (<5k)", "Mid (5k-10k)", "Upper-mid (10k-20k)", "Premium (20k+)"]
df["price_bin"] = pd.cut(df["y_true"], bins=bins, labels=labels)

# Aggregate error by bin
df.groupby("price_bin")[["error", "perc_error"]].mean()

,error,perc_error
price_bin,,
Budget (<5k),360.634088,0.094753
Mid (5k-10k),754.881469,0.105751
Upper-mid (10k-20k),829.596044,0.062192
Premium (20k+),6752.998708,0.193940


In [63]:
print("Total rows below 5000 are",y_train.to_frame().query('price < 5000').shape[0])
print("Total rows between 5000 to 10000 are",y_train.to_frame().query('price < 10000 & price > 5000').shape[0])
print("Total rows between 10000 to 20000 are",y_train.to_frame().query('price < 20000 & price > 10000').shape[0])
print("Total rows above 20000 are",y_train.to_frame().query('price > 20000').shape[0])
print("\nTotal rows are" ,y_train.shape[0])

Total rows below 5000 are 1533
Total rows between 5000 to 10000 are 2470
Total rows between 10000 to 20000 are 2592
Total rows above 20000 are 94

Total rows are 6694


#### Applying the MOE concept

In [32]:
train_df = X_train_transformed.copy()
val_df = X_val_transformed.copy()
train_df['price'] = y_train.values 
val_df['price'] = y_val.values
train_df_low = train_df.query('price < 10000')
train_df_high = train_df.query('price >10000 & price < 20000')
X_train_low = train_df_low.drop(columns=['price'])
y_train_low = train_df_low[['price']]
X_train_high = train_df_high.drop(columns=['price'])
y_train_high = train_df_high[['price']]

val_df_low = val_df.query('price < 10000')
val_df_high = val_df.query('price >10000 & price < 20000') 
X_val_low = val_df_low.drop(columns=['price'])
y_val_low = val_df_low[['price']]
X_val_high = val_df_high.drop(columns=['price'])
y_val_high = val_df_high[['price']]

In [ ]:
# Tuning the Expert_1 model

import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib  # for saving model

def objective(trial):
    # Suggest hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "random_state": 42,
        "n_jobs": -1
    }
    
    model = XGBRegressor(**param)
    model.fit(X_train_low, y_train_low)
    preds = model.predict(X_val_low)
    mse = mean_squared_error(y_val_low, preds)
    return mse  # Minimize MSE


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50) 

[I 2026-01-12 20:14:09,470] A new study created in memory with name: no-name-4f5f761d-1152-40de-8fc4-831bed640e4d
[I 2026-01-12 20:14:14,424] Trial 0 finished with value: 576097.4375 and parameters: {'n_estimators': 742, 'max_depth': 16, 'learning_rate': 0.20427323440123582, 'subsample': 0.9417945940724376, 'colsample_bytree': 0.9450525488692024, 'gamma': 4.534625639734652, 'min_child_weight': 7, 'reg_alpha': 9.41564656910026, 'reg_lambda': 6.234178867272044}. Best is trial 0 with value: 576097.4375.
[I 2026-01-12 20:14:17,544] Trial 1 finished with value: 549030.3125 and parameters: {'n_estimators': 911, 'max_depth': 10, 'learning_rate': 0.12816808751166, 'subsample': 0.6248343476556679, 'colsample_bytree': 0.5386423254402419, 'gamma': 8.121827096955075, 'min_child_weight': 3, 'reg_alpha': 4.686248516120984, 'reg_lambda': 7.622928703173843}. Best is trial 1 with value: 549030.3125.
[I 2026-01-12 20:14:17,833] Trial 2 finished with value: 540799.125 and parameters: {'n_estimators': 282

In [34]:
# Expert_model_1

best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

expert_model_1 = XGBRegressor(**best_params)
expert_model_1.fit(X_train_low, y_train_low)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8409369333076588, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=6.51481807476888, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06284122962053267,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=444, n_jobs=-1,
             num_parallel_tree=None, ...)

In [35]:
# Tuning the Expert_2 model

import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import joblib  # for saving model

def objective(trial):
    # Suggest hyperparameters
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
        "random_state": 42,
        "n_jobs": -1
    }

    # Model
    model = XGBRegressor(**param)

    # Fit on training data
    model.fit(X_train_high, y_train_high)

    # Predict on tuning set
    preds = model.predict(X_val_high)

    # Evaluation metric
    mse = mean_squared_error(y_val_high, preds)
    return mse  # Minimize MSE


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50) 

[I 2026-01-12 20:15:26,662] A new study created in memory with name: no-name-ead716ea-fe52-4e59-8370-565d9b043e4e
[I 2026-01-12 20:15:27,364] Trial 0 finished with value: 1154562.75 and parameters: {'n_estimators': 113, 'max_depth': 4, 'learning_rate': 0.29759960174603, 'subsample': 0.8815400075626281, 'colsample_bytree': 0.7614012302177944, 'gamma': 1.2756384487066152, 'min_child_weight': 7, 'reg_alpha': 4.762601057821792, 'reg_lambda': 5.6194997294399816}. Best is trial 0 with value: 1154562.75.
[I 2026-01-12 20:15:39,535] Trial 1 finished with value: 901838.4375 and parameters: {'n_estimators': 960, 'max_depth': 17, 'learning_rate': 0.013092280825756074, 'subsample': 0.7907212658791374, 'colsample_bytree': 0.9532883014258084, 'gamma': 5.508406095501119, 'min_child_weight': 1, 'reg_alpha': 5.404211808171887, 'reg_lambda': 2.672608139846827}. Best is trial 1 with value: 901838.4375.
[I 2026-01-12 20:15:40,047] Trial 2 finished with value: 1006552.0 and parameters: {'n_estimators': 134

In [36]:
# Expert_model_2

best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

expert_model_2 = XGBRegressor(**best_params)
expert_model_2.fit(X_train_high, y_train_high)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8561312653807508, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=6.617814517684063, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.010948469694003577,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=18, max_leaves=None,
             min_child_weight=5, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=661, n_jobs=-1,
             num_parallel_tree=None, ...)

In [37]:
# Making the gate 

from xgboost import XGBClassifier

y_cluster = (y_train < 10000).astype(int)  # 1 if y_train<10k, else 0

gating = XGBClassifier(n_estimators=200, max_depth=4)
gating.fit(X_train_transformed, y_cluster)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=200,
              n_jobs=None, num_parallel_tree=None, ...)

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Predict on validation / test
y_gate_pred = gating.predict(X_train_transformed)

# Metrics
print("Accuracy:", accuracy_score(y_cluster, y_gate_pred))
print("Precision:", precision_score(y_cluster, y_gate_pred))
print("Recall:", recall_score(y_cluster, y_gate_pred))
print("F1:", f1_score(y_cluster, y_gate_pred))

Accuracy: 0.9757992231849417
Precision: 0.9785963165754107
Recall: 0.9810379241516967
F1: 0.9798155993022676


In [39]:
import numpy as np

def moe_predict(X):
    # step 1: gating probs
    probs = gating.predict_proba(X)  # shape (n_samples, 2)
    p_low = probs[:, 1]   # prob cluster=low
    p_high = probs[:, 0]  # prob cluster=high
    
    # step 2: expert predictions
    pred_low = expert_model_1.predict(X)
    pred_high = expert_model_2.predict(X)
    
    # step 3: weighted combination
    pred_final = p_low * pred_low + p_high * pred_high

    return pred_final


In [40]:
y_test_pred_moe = moe_predict(X_test_transformed)
rmse_moe = np.sqrt(mean_squared_error(y_test, y_test_pred_moe))
mae_moe = mean_absolute_error(y_test, y_test_pred_moe)
r2_moe = r2_score(y_test, y_test_pred_moe)
print("Rmse score",rmse_moe)
print("mae score",mae_moe)
print("R2 score",r2_moe)

Rmse score 2344.5204200432977
mae score 853.0096435546875
R2 score 0.7643895149230957


In [41]:
df_moe = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_test_pred_moe
})

# 3. Compute absolute + percentage error
df_moe["error"] = abs(df_moe["y_true"] - df_moe["y_pred"])
df_moe["perc_error"] = df_moe["error"] / df_moe["y_true"]

# 4. Define bins
bins = [0, 5000, 10000, 20000, 80000]
labels = ["Budget (<5k)", "Mid (5k-10k)", "Upper-mid (10k-20k)", "Premium (20k+)"]

df_moe["price_bin"] = pd.cut(df_moe["y_true"], bins=bins, labels=labels)

# 5. Aggregate error by bin
df_moe.groupby("price_bin")[["error", "perc_error"]].mean()



,error,perc_error
price_bin,,
Budget (<5k),336.966552,0.087707
Mid (5k-10k),765.095877,0.107172
Upper-mid (10k-20k),768.806355,0.057786
Premium (20k+),13163.426254,0.398451


### Taking the Random Forest model

In [20]:
# Taking the Random Forest model

import optuna
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def objective(trial):
    param = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),  # ✅ fixed
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "random_state": 42,
        "n_jobs": -1
    }

    model = RandomForestRegressor(**param)
    model.fit(X_train_transformed, y_train)

    preds = model.predict(X_val_transformed)
    mse = mean_squared_error(y_val, preds)

    return mse

# Run the study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)


[I 2025-08-22 17:34:30,960] A new study created in memory with name: no-name-d582bb2e-f07f-4473-a974-2ed9575b893a
[I 2025-08-22 17:34:32,420] Trial 0 finished with value: 3949740.988178595 and parameters: {'n_estimators': 332, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': 'log2', 'bootstrap': True}. Best is trial 0 with value: 3949740.988178595.
[I 2025-08-22 17:34:35,092] Trial 1 finished with value: 3235487.863368827 and parameters: {'n_estimators': 848, 'max_depth': 17, 'min_samples_split': 15, 'min_samples_leaf': 5, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 1 with value: 3235487.863368827.
[I 2025-08-22 17:34:36,873] Trial 2 finished with value: 3487995.696783186 and parameters: {'n_estimators': 602, 'max_depth': 18, 'min_samples_split': 13, 'min_samples_leaf': 8, 'max_features': 'sqrt', 'bootstrap': True}. Best is trial 1 with value: 3235487.863368827.
[I 2025-08-22 17:34:38,152] Trial 3 finished with value: 2945370.4606393795 and p

In [21]:
best_params = study.best_params
best_params["random_state"] = 42
best_params["n_jobs"] = -1

final_rf_model = RandomForestRegressor(**best_params)
final_rf_model.fit(X_train_transformed, y_train)

model_dir = os.path.join("..", "artifacts", "models")
os.makedirs(model_dir, exist_ok=True)

# Full save path for model
model_path = os.path.join(model_dir, "randomforest_flight_price_model.joblib")

# Save the model
joblib.dump(final_rf_model, model_path)

print(f"✅ Final model trained and saved as {model_path}")

✅ Final model trained and saved as ..\artifacts\models\randomforest_flight_price_model.joblib


In [22]:
from sklearn.metrics import r2_score, mean_absolute_error

y_pred = final_rf_model.predict(X_test_transformed)

print("Test MSE:", mean_squared_error(y_test, y_pred))
print("Test MAE:", mean_absolute_error(y_test, y_pred))
print("Test R²:", r2_score(y_test, y_pred))

Test MSE: 3620046.3503122544
Test MAE: 974.8438947050673
Test R²: 0.8355106881602943


In [23]:
y_pred = final_rf_model.predict(X_train_transformed)

print("Train MSE:", mean_squared_error(y_train, y_pred))
print("Train MAE:", mean_absolute_error(y_train, y_pred))
print("Train R²:", r2_score(y_train, y_pred))

Train MSE: 1488026.9397696452
Train MAE: 629.9009802370945
Train R²: 0.93199710266873
